In [6]:
from dotenv import load_dotenv
load_dotenv()
import os, json, pickle, time
import regex as re
from bot import set_driver, open_page, get_href_page, check_page
from parse import get_article_info
import pandas as pd

output_name = "test"

#input_company_file = "STEP4_Factiva_company_list_7551_7600_sector_Haokun Zhang.csv"

ut_eid = "hz7297"

eid_password = os.getenv('eid_password')

driver_path = "C://Users//Haokun Zhang//chromedriver//chromedriver.exe"

ARTICLE_DIR = os.path.join('..//data', 'article_hrefs', 'test')

os.makedirs(ARTICLE_DIR, exist_ok=True)

In [7]:
def gen_searches():
    
    try:
        with open('step5_searches.pickle', 'rb') as file:
            articles = pickle.load(file)
            if len(articles) == 0:
                print('Create a new pickle to manage searches; current list has been exhausted.')
    except FileNotFoundError:
        articles = {}
        for file_name in os.listdir(ARTICLE_DIR):
            l = []
            with open(os.path.join(os.path.join(ARTICLE_DIR, file_name))) as file:
                href_list =  [t for t in re.sub('\n','',file.read()).split(r'"') if len(re.findall('\w+', t)) > 0]
                for href in href_list:
                    l.append(f"https://global-factiva-com.ezproxy.lib.utexas.edu/{href.split('&')[0].split('../')[1]}")
            articles[file_name.split('.')[0]] = l
    return articles



In [8]:
def search_article_list(driver, wait, article_list, name):
    
    temp_list = []
    name_list = name.split('_')
    for href in article_list:
        temp_dict = {'company_code': name_list[0], 'news_code': name_list[1], 'date': name_list[2]}
        get_href_page(driver, wait, href)
        check_page(driver, wait, href)
        temp_list.append(get_article_info(driver.page_source, temp_dict))
        time.sleep(4)
    return temp_list

In [9]:
def get_text(eid_username, eid_password, path):
    
    results = []
    searches = gen_searches()
    searches_pickle = searches.copy()
    driver, wait = set_driver(path)
    open_page(driver, wait, eid_username, eid_password)
    
    for name, article_list in searches.items():
        for i, (key, item) in enumerate('step5_searches.pickle'):
            if item % 10 == 0:
                break
            else:
                continue
        temp_list = search_article_list(driver, wait, article_list, name)
        if temp_list != None:
            results += temp_list
            print(results)
        try:
            searches_pickle.pop(name)
            with open('step5_searches.pickle', 'wb') as file:
                pickle.dump(searches_pickle, file)
            with open('dict_list.pickle', 'wb') as file:
                pickle.dump(results, file)
        except:
            return 'Error when saving'
        
    pd.DataFrame.from_dict(results).to_csv('article_test.csv')

In [10]:
get_text(ut_eid, eid_password, driver_path)

Create a new pickle to manage searches; current list has been exhausted.
